# TPE_Twin-Delayed DDPG

## Installation des paquets

In [1]:
!pip install pybullet

     |████████████████████████████████| 102.2MB 64kB/s 


#Importation des librairies


In [2]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

## Etape 1: On itinialise Experience Replay memory




In [3]:
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind: 
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

## Etape 2: Nous construisons un réseau neuronal pour le modèle Actor et un réseau neuronal pour la cible Actor

In [4]:
class Actor(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x))
    return x

## Étape 3 : Nous construisons deux réseaux de neurones pour les deux modèles Critic et deux réseaux de neurones pour les deux cibles Critic

In [5]:
class Critic(nn.Module):
  
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # Définir le premier réseau neuronal critique
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # Définir le second réseau neuronal critique
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    # Propagation de l'information sur le premier réseau neuronal critique
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    # Propagation de l'information sur le second réseau neuronal critique
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2

  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

## Étapes 4 à 15 : Processus de d'entraînement

In [6]:
#  Device selection (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Intégrer l'ensemble du processus de formation dans une classe

class TD3(object):
  
  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      # Étape 4: Nous échantillonnons un lot de transitions (s, s’, a, r) de la mémoire

      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      # Étape 5: De l'état suivant s', l'acteur cible joue l'action suivante a’
      next_action = self.actor_target(next_state)
      
      # Étape 6: Nous ajoutons le bruit gaussien à cette prochaine action a’ et nous l'inscrivons dans une série de valeurs soutenues par l'environnement
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      # Étape 7: Les deux cibles des critiques prennent chacune le couple (s’, a’) comme input et retourne deux Q-values Qt1(s’,a’) et Qt2(s’,a’) comme outputs
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      
      # Étape 8: Nous gardons le minimum de ces deux Q-values: min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)
      
      # Étape 9: Nous obtenons la cible finale des deux modèles critiques, qui est: Qt = r + γ * min(Qt1, Qt2), où γ est le coefficient d'actualisation
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      # Étape 10: Les deux modèles critiques prennent chacun le couple (s, a) comme input et retourne deux Q-values Q1(s,a) et Q2(s,a) comme outputs
      current_Q1, current_Q2 = self.critic(state, action)
      
      # Étape 11: Nous calculons la perte à partir des deux modèles critiques: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      
      # Étape 12: Nous rétropropageons cette perte Critic et mettons à jour les paramètres des deux modèles Critic avec un optimiseur SGD
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      
      # Étape 13:  Une fois pour chaque deux itérations, nous mettons à jour notre modèle Acteur en effectuant une montée du gradient sur la sortie du premier modèle Critique
      if it % policy_freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Étape 14: Une fois pour chaque deux itérations, nous mettons à jour les poids de l'acteur cible en calculant la moyenne des polyak
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        # Étape 15 : Une fois pour chaque deux itérations, nous mettons à jour les poids de la cible critique en calculant la moyenne des polyak
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  # Faire une méthode de sauvegarde pour sauvegarder un modèle entrainé
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Mise au point d'une méthode de chargement pour charger un modèle pré-entrainé
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

## Nous faisons une fonction qui évalue la politique en calculant sa récompense moyenne sur 10 épisodes

In [7]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward




```
# Ce texte est au format code
```

## Nous fixons les paramètres

In [8]:
env_name = "HalfCheetahBulletEnv-v0" # Nom de l'environnement (Nous le reglons sur environnement continu que nous souhaitons)
seed = 0 # Numéro de fixation(seed) aléatoire
start_timesteps = 1e4 # Nombre d'itérations / étapes avant lesquelles le modèle choisit une action au hasard, et après lesquelles il commence à utiliser le réseau politique
eval_freq = 5e3 # Combien de fois l'étape d'évaluation est effectuée (après combien de temps)
max_timesteps = 5e5 # Nombre total d'itérations / étapes
save_models = True # Vérificateur booléen pour savoir s'il faut ou non sauver le modèle préformé
expl_noise = 0.1 # Bruit d'exploration - Valeur STD de l'exploration Bruit gaussien
batch_size = 100 # Taille du lot
discount = 0.99 # Facteur d'escompte gamma, utilisé dans le calcul de la récompense totale escomptée
tau = 0.005 # Taux de mise à jour du réseau cible
policy_noise = 0.2 # MST du bruit gaussien ajoutée aux actions à des fins d'exploration
noise_clip = 0.5 # Valeur maximale du bruit gaussien ajouté aux actions (politique)
policy_freq = 2 # Nombre d'itérations à attendre avant de mettre à jour le réseau politique (modèle Actor)

## Nous créons un nom de fichier pour les deux modèles sauvegardés : les modèles Acteur et Critique

In [9]:
file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

---------------------------------------
Settings: TD3_HalfCheetahBulletEnv-v0_0
---------------------------------------


## Nous créons un dossier à l'intérieur duquel seront sauvegardés les modèles formés

In [10]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

## Nous créons l'environnement PyBullet

In [11]:
env = gym.make(env_name)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Nous mettons en place les valeurs et nous obtenons les informations nécessaires sur les états et les actions dans l'environnement choisi

In [12]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

## Nous créons le réseau politique (le modèle Actor)

In [13]:
policy = TD3(state_dim, action_dim, max_action)

## Nous créons la mémoire de l'expérience Replay

In [14]:
replay_buffer = ReplayBuffer()

## Nous définissons une liste où sont stockés tous les résultats d'évaluation sur 10 épisodes

In [15]:
evaluations = [evaluate_policy(policy)]

---------------------------------------
Average Reward over the Evaluation Step: -1429.426163
---------------------------------------


## Nous créons un nouveau répertoire de dossiers dans lequel les résultats finaux (vidéos de l'agent) seront remplis

In [16]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path
work_dir = mkdir('exp', 'brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps
save_env_vid = False
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()

## Nous initialisons les variables

In [17]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

## Entrainement

In [18]:
# Nous démarrons la boucle principale en 500 000 pas
while total_timesteps < max_timesteps:
  
  # Si l'épisode est terminé
  if done:

    # Si nous ne sommes pas au tout début, nous commençons le processus de formation du modèle
    if total_timesteps != 0:
      print("Total Timesteps: {} Episode Num: {} Reward: {}".format(total_timesteps, episode_num, episode_reward))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    #  Nous évaluons l'épisode et nous sauvegardons la politique
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      policy.save(file_name, directory="./pytorch_models")
      np.save("./results/%s" % (file_name), evaluations)
    
    # Lorsque l'étape de l'entrainement est terminée, nous réinitialisons l'état de l'environnement

    obs = env.reset()
    
    # Mettre Done à False (Si l'épisode n'est pas terminé)
    done = False
    
    # Mettre à zéro les récompenses et les pas de temps des épisodes
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Avant 10000 pas de temps, nous jouons à des actions aléatoires
  if total_timesteps < start_timesteps:
    action = env.action_space.sample()
  else: # Après 10 000 pas, nous changeons de modèle
    action = policy.select_action(np.array(obs))
    # Si le paramètre explore_noise n'est pas 0, nous ajoutons du bruit à l'action et nous la découpons
    if expl_noise != 0:
      action = (action + np.random.normal(0, expl_noise, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
  
  # L'agent effectue l'action dans l'environnement, puis passe à l'état suivant et reçoit la récompense
  new_obs, reward, done, _ = env.step(action)
  
  # Nous vérifions si l'épisode est terminé
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
  
  # Nous augmentons la récompense totale
  episode_reward += reward
  
  # Nous enregistrons la nouvelle transition dans Experience Replay memory (ReplayBuffer)
  replay_buffer.add((obs, new_obs, action, reward, done_bool))

  # Nous mettons à jour l'état, la durée de l'épisode, la durée totale et la durée depuis l'évaluation de la politique
  obs = new_obs
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# Nous ajoutons la dernière évaluation de la politique à notre liste d'évaluations et nous sauvegardons notre modèle
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)

Total Timesteps: 1000 Episode Num: 1 Reward: -1428.9683875246344
Total Timesteps: 2000 Episode Num: 2 Reward: -1346.3968009963335
Total Timesteps: 3000 Episode Num: 3 Reward: -1454.0231639096664
Total Timesteps: 4000 Episode Num: 4 Reward: -1366.327901297108
Total Timesteps: 5000 Episode Num: 5 Reward: -1167.8386884896897
---------------------------------------
Average Reward over the Evaluation Step: -1495.642147
---------------------------------------
Total Timesteps: 6000 Episode Num: 6 Reward: -1262.1999878148674
Total Timesteps: 7000 Episode Num: 7 Reward: -1137.836346134204
Total Timesteps: 8000 Episode Num: 8 Reward: -1282.186879642088
Total Timesteps: 9000 Episode Num: 9 Reward: -1268.0734087566348
Total Timesteps: 10000 Episode Num: 10 Reward: -1414.2107823890558
---------------------------------------
Average Reward over the Evaluation Step: -1796.337328
---------------------------------------
Total Timesteps: 11000 Episode Num: 11 Reward: -1733.5919930837288
Total Timesteps:

## Inférence

In [27]:
class Actor(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x)) 
    return x

class Critic(nn.Module):
  
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # Définir le premier réseau neuronal critique
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # Définir le second réseau neuronal critique
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    # Propagation de l'information sur le premier réseau neural critique
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    # Propagation de l'information sur le deuxième réseau neuronal critique
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2

  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

# Device selection (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Intégrer l'ensemble du processus de formation dans une classe

class TD3(object):
  
  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      # Étape 4: Nous échantillonnons un lot de transitions (s, s’, a, r) de la mémoire
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      # Étape 5: De l'état suivant s', l'acteur cible joue l'action suivante a’
      next_action = self.actor_target(next_state)
      
      # Étape 6: Nous ajoutons le bruit gaussien à cette prochaine action a’ et nous l'inscrivons dans une série de valeurs soutenues par l'environnement
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      # Étape 7: Les deux cibles des critiques prennent chacune le couple (s’, a’) comme input et retourne deux Q-values Qt1(s’,a’) et Qt2(s’,a’) comme outputs
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      
      # Étape 8: Nous gardons le minimum de ces deux Q-values: min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)
      
      # Étape 9: Nous obtenons la cible finale des deux modèles critiques, qui est: Qt = r + γ * min(Qt1, Qt2), où γ est le coefficient d'actualisation
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      # Étape 10: Les deux modèles critiques prennent chacun le couple (s, a) comme input et retourne deux Q-values Q1(s,a) et Q2(s,a) comme outputs
      current_Q1, current_Q2 = self.critic(state, action)
      
      # Étape 11: Nous calculons la perte à partir des deux modèles critiques: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      
      # Étape 12: Nous rétropropageons cette perte Critic et mettons à jour les paramètres des deux modèles Critic avec un optimiseur SGD
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      
      # Étape 13:  Une fois pour chaque deux itérations, nous mettons à jour notre modèle Acteur en effectuant une montée du gradient sur la sortie du premier modèle Critique
      if it % policy_freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Étape 14: Une fois pour chaque deux itérations, nous mettons à jour les poids de l'acteur cible en calculant la moyenne des polyak
#  Still once every two iterations, we update the weights of the Actor target by polyak averaging
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        # Étape 15 : Une fois pour chaque deux itérations, nous mettons à jour les poids de la cible critique en calculant la moyenne des polyak
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  # Faire une méthode de sauvegarde pour sauvegarder un modèle entraîné
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Mise au point d'une méthode de chargement pour charger un modèle pré-entrainé
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

env_name = "HalfCheetahBulletEnv-v0"
seed = 0

file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

eval_episodes = 10
save_env_vid = True
env = gym.make(env_name)
max_episode_steps = env._max_episode_steps
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
policy = TD3(state_dim, action_dim, max_action)
policy.load(file_name, './pytorch_models/')
_ = evaluate_policy(policy, eval_episodes=eval_episodes)

---------------------------------------
Settings: TD3_HalfCheetahBulletEnv-v0_0
---------------------------------------


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------------
Average Reward over the Evaluation Step: 2698.533188
---------------------------------------
